In [68]:
# imports
import cv2
from pathlib import Path, PosixPath
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
import natsort

In [58]:
def get_files(caminho_p):
    i=0
    train, test = list(), list()
    
    for pasta in sorted(Path(caminho_p).iterdir()):
      if pasta.is_dir(): 
        for file in sorted(Path(pasta).iterdir()):
            if file.is_file() and file.name.endswith(".bmp"):
              num = file.name.split(".")[0].split("_")[1]
              if int(num) > 18:
                test.append(str(file.absolute()))
              else:
                train.append(str(file.absolute()))
              i+=1
    print(f"numero de arquivos: ", i)
    return train, test
  
def get_classe(x):
  classe = list()
  for i in x:
    file = PosixPath(i)
    classe.append(file.name.split("_")[0])
  return classe

In [59]:
#https://drive.google.com/drive/folders/1-3DJsfgM73-AHlkjn3-VUedZWnmllCud?usp=drive_link
train, test = get_files("../Base_BFL_256/")
train = natsort.natsorted(train) 
test = natsort.natsorted(test)
classes_train = get_classe(train)
classes_test = get_classe(test)
print(classes_train)
print(train)
print(test)
print(f"tamanho de train {len(train)}")
print(f"tamanho de test {len(test)}")
print(f"tamanho de classes train {len(classes_train)}")
print(f"tamanho de classes test {len(classes_test)}")


numero de arquivos:  270
['CF00001', 'CF00001', 'CF00001', 'CF00001', 'CF00001', 'CF00001', 'CF00001', 'CF00001', 'CF00001', 'CF00001', 'CF00001', 'CF00001', 'CF00001', 'CF00001', 'CF00001', 'CF00001', 'CF00001', 'CF00001', 'CF00002', 'CF00002', 'CF00002', 'CF00002', 'CF00002', 'CF00002', 'CF00002', 'CF00002', 'CF00002', 'CF00002', 'CF00002', 'CF00002', 'CF00002', 'CF00002', 'CF00002', 'CF00002', 'CF00002', 'CF00002', 'CF00003', 'CF00003', 'CF00003', 'CF00003', 'CF00003', 'CF00003', 'CF00003', 'CF00003', 'CF00003', 'CF00003', 'CF00003', 'CF00003', 'CF00003', 'CF00003', 'CF00003', 'CF00003', 'CF00003', 'CF00003', 'CF00004', 'CF00004', 'CF00004', 'CF00004', 'CF00004', 'CF00004', 'CF00004', 'CF00004', 'CF00004', 'CF00004', 'CF00004', 'CF00004', 'CF00004', 'CF00004', 'CF00004', 'CF00004', 'CF00004', 'CF00004', 'CF00005', 'CF00005', 'CF00005', 'CF00005', 'CF00005', 'CF00005', 'CF00005', 'CF00005', 'CF00005', 'CF00005', 'CF00005', 'CF00005', 'CF00005', 'CF00005', 'CF00005', 'CF00005', 'CF000

In [63]:
# Função para calcular o LBP de uma imagem
def calculate_lbp(imagefile):
    image = cv2.imread(str(imagefile))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Converte a imagem para tons de cinza
    lbp_image = np.zeros_like(gray) # Cria uma matriz de zeros com as mesmas dimensões da imagem
    height, width = gray.shape

    for i in range(1, height - 1):
        for j in range(1, width - 1):
            center = gray[i, j]
            binary_code = 0
            binary_code |= (gray[i - 1, j - 1] > center) << 7
            binary_code |= (gray[i - 1, j] > center) << 6
            binary_code |= (gray[i - 1, j + 1] > center) << 5
            binary_code |= (gray[i, j + 1] > center) << 4
            binary_code |= (gray[i + 1, j + 1] > center) << 3
            binary_code |= (gray[i + 1, j] > center) << 2
            binary_code |= (gray[i + 1, j - 1] > center) << 1
            binary_code |= (gray[i, j - 1] > center) << 0
            lbp_image[i, j] = binary_code

    return lbp_image


In [61]:
def knn(neighbors, train, classes, test):

    knn = KNeighborsClassifier(n_neighbors=neighbors, metric='euclidean')
    knn.fit(train, classes)
    test_pred = knn.predict(test)
    return test_pred


In [64]:
lbp_train = list()
for imagefile in train:
    lbp = calculate_lbp(imagefile)
    lbp_train.append(lbp.ravel())

lbp_test = list()
for imagefile in test:
    lbp = calculate_lbp(imagefile)
    lbp_test.append(lbp.ravel())

In [65]:

test_pred = knn(3, lbp_train, classes_train, lbp_test)
print(test_pred)
print(classes_test)

['CF00005' 'CF00008' 'CF00005' 'CF00008' 'CF00005' 'CF00008' 'CF00008'
 'CF00008' 'CF00008' 'CF00008' 'CF00008' 'CF00008' 'CF00008' 'CF00008'
 'CF00008' 'CF00008' 'CF00008' 'CF00008' 'CF00008' 'CF00008' 'CF00008'
 'CF00008' 'CF00008' 'CF00008' 'CF00005' 'CF00008' 'CF00008' 'CF00005'
 'CF00005' 'CF00005' 'CF00005' 'CF00005' 'CF00005' 'CF00005' 'CF00005'
 'CF00005' 'CF00005' 'CF00005' 'CF00005' 'CF00005' 'CF00005' 'CF00005'
 'CF00005' 'CF00005' 'CF00005' 'CF00005' 'CF00005' 'CF00005' 'CF00005'
 'CF00005' 'CF00005' 'CF00005' 'CF00005' 'CF00005' 'CF00005' 'CF00008'
 'CF00005' 'CF00008' 'CF00005' 'CF00005' 'CF00008' 'CF00008' 'CF00008'
 'CF00005' 'CF00008' 'CF00005' 'CF00008' 'CF00005' 'CF00008' 'CF00008'
 'CF00008' 'CF00005' 'CF00005' 'CF00005' 'CF00005' 'CF00005' 'CF00005'
 'CF00005' 'CF00005' 'CF00005' 'CF00005' 'CF00005' 'CF00005' 'CF00005'
 'CF00008' 'CF00005' 'CF00005' 'CF00005' 'CF00005' 'CF00005']
['CF00001', 'CF00001', 'CF00001', 'CF00001', 'CF00001', 'CF00001', 'CF00001', 'CF00001

In [70]:
print(accuracy_score(classes_test, test_pred))
print(f1_score(classes_test, test_pred, average='macro'))

0.15555555555555556
0.05094812164579606


In [ ]:
# Calcular o LBP da imagem
lbp = calculate_lbp('../Base_BFL_256/CF00001/CF00001_1.bmp')

lbp_df = pd.DataFrame(lbp)
print(np.array(lbp.flatten()))
#escrever em arquivo
with open('lbp.txt', 'w') as f:
    # for item in lbp.flatten():
        f.write("%s\n" % lbp_df.to_string(index=False, header=False))

lbp_df

# Mostrar a imagem LBP
cv2.imshow('LBP Image', lbp)
cv2.waitKey(0)
cv2.destroyAllWindows()

[0 0 0 ... 0 0 0]


,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,225,247,224,231,255,0,65,175,0
2,0,0,0,0,0,0,0,0,0,195,...,227,192,171,223,142,191,126,198,139,0
3,0,0,227,225,249,248,0,0,195,195,...,239,207,223,30,14,31,26,4,223,0
4,0,0,227,253,254,120,0,0,199,203,...,158,30,28,14,31,6,127,4,143,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,0,0,0,0,0,0,0,0,0,0,...,255,64,249,12,159,6,135,231,255,0
252,0,0,0,0,0,0,0,0,0,0,...,191,126,252,0,0,7,135,135,191,0
253,0,0,0,0,0,0,0,0,0,0,...,7,63,60,0,0,135,135,135,167,0
254,0,0,0,0,0,0,0,0,0,195,...,143,191,0,0,0,0,135,135,135,0
